<a href="https://colab.research.google.com/github/tsoetan/tsoetan/blob/main/ping_tool_for_teaching_security.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================
# SYSTEM UTILITY: PING TOOL (VULNERABLE)
# ==========================================
import subprocess

def check_server(ip_address):
    print(f"Checking connectivity to: {ip_address}")

    # SYSTEM COMMAND execution
    # Vulnerability: Directly passing user input to the shell
    command = f"ping -c 1 {ip_address}"

    try:
        # 'shell=True' is often used by beginners (and AI) to make things "just work"
        output = subprocess.check_output(command, shell=True, text=True)
        return "Server is UP"
    except subprocess.CalledProcessError:
        return "Server is DOWN"

# SIMULATE A HACKER INPUT
# The user enters an IP, but adds a semicolon to run a second command.
user_input = "8.8.8.8; echo 'I HAVE STOLEN YOUR PASSWORDS'"

print(check_server(user_input))

In [ ]:
#the fix

# ==========================================
# SYSTEM UTILITY: PING TOOL (SECURE)
# ==========================================
import subprocess
import re
import ipaddress

def check_server(ip_address):
    """
    Safely checks server connectivity by pinging an IP address.

    Args:
        ip_address: String containing an IP address

    Returns:
        String indicating server status or error message
    """
    print(f"Checking connectivity to: {ip_address}")

    # Step 1: Validate and sanitize input
    try:
        # Strip whitespace
        ip_address = ip_address.strip()

        # Validate it's a proper IP address (IPv4 or IPv6)
        # This will raise ValueError if invalid
        ipaddress.ip_address(ip_address)

    except ValueError:
        return f"ERROR: Invalid IP address format: '{ip_address}'"

    # Step 2: Execute ping safely
    try:
        # SECURITY FIX: Use list format with shell=False (default)
        # This prevents command injection by treating each argument separately
        result = subprocess.run(
            ["ping", "-c", "1", "-W", "2", ip_address],
            capture_output=True,
            text=True,
            timeout=5  # Prevent hanging
        )

        # Check return code
        if result.returncode == 0:
            return "Server is UP"
        else:
            return "Server is DOWN"

    except subprocess.TimeoutExpired:
        return f"ERROR: Ping timeout for {ip_address}"
    except FileNotFoundError:
        return "ERROR: 'ping' command not found on this system"
    except Exception as e:
        return f"ERROR: Unexpected error occurred: {str(e)}"


# Test cases
print("=" * 50)
print("Testing valid IP:")
print(check_server("8.8.8.8"))

print("\n" + "=" * 50)
print("Testing malicious input (BLOCKED):")
print(check_server("8.8.8.8; echo 'I HAVE STOLEN YOUR PASSWORDS'"))

print("\n" + "=" * 50)
print("Testing other attack vectors (BLOCKED):")
print(check_server("8.8.8.8 && cat /etc/passwd"))
print(check_server("8.8.8.8 | rm -rf /"))
print(check_server("'; DROP TABLE users; --"))

print("\n" + "=" * 50)
print("Testing invalid inputs:")
print(check_server("999.999.999.999"))
print(check_server("not-an-ip"))
print(check_server(""))
